<a href="https://colab.research.google.com/github/AlphaAzim/streamlit-example/blob/master/Stock_AN_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import yfinance as yf
import streamlit as st
import matplotlib.pyplot as plt

def get_realtime_data(ticker_symbols):
    stock_data = {}
    for symbol in ticker_symbols:
        stock = yf.Ticker(symbol)
        info = stock.info

        stock_data[symbol] = {
            "Price": info.get("regularMarketPrice", "N/A"),
            "EPS": info.get("trailingEps", "N/A"),
            "PE Ratio": info.get("trailingPE", "N/A"),
            "Profit Margin": info.get("profitMargins", "N/A"),
            "Market Cap": info.get("marketCap", "N/A"),
            "Bid": info.get("bid", "N/A"),
            "Ask": info.get("ask", "N/A"),
            "Open": info.get("regularMarketOpen", "N/A"),
            "Close": info.get("regularMarketPreviousClose", "N/A"),
            "52 Week Range": info.get("fiftyTwoWeekLow", "N/A"),
            "Volume": info.get("regularMarketVolume", "N/A"),
            "Avg Volume": info.get("averageDailyVolume10Day", "N/A"),
        }

    return stock_data

def plot_price_graph(symbol, show_ma=False):
    stock = yf.Ticker(symbol)
    df = stock.history(period="1d", interval="1m")

    if len(df) >= 2:
        fig, ax = plt.subplots(figsize=(12, 6))
        plt.subplots_adjust(bottom=0.2)
        line, = ax.plot(df.index, df["Close"], label=f"{symbol} Price", color="b")
        plt.xlabel("Time")
        plt.ylabel("Price (USD)")
        plt.title(f"{symbol} Stock Price")
        ax.legend()

        if show_ma:
            ma_20 = df["Close"].rolling(window=20).mean()
            ma_50 = df["Close"].rolling(window=50).mean()
            ax.plot(df.index, ma_20, label="20-day MA", color="g")
            ax.plot(df.index, ma_50, label="50-day MA", color="r")

        plt.grid(True)

        # Add labels for moving averages to the legend
        if show_ma:
            handles, labels = ax.get_legend_handles_labels()
            labels = [f"{label} ({handle.get_color()})" for handle, label in zip(handles, labels)]
            ax.legend(handles, labels)

        # Show the plot using Streamlit's st.pyplot() function
        st.pyplot(fig)

        # Close the plot to avoid potential memory leaks
        plt.close()

    else:
        st.write(f"Not enough data available for {symbol} to plot the price graph.")

def analyze_data(data):
    analysis = {}
    for symbol, stock_data in data.items():
        if stock_data["Price"] == "N/A" or stock_data["Close"] == "N/A":
            analysis[symbol] = "Not enough data for analysis"
        else:
            if stock_data["Price"] > stock_data["Close"]:
                analysis[symbol] = "Recommendation: Buy"
            elif stock_data["Price"] < stock_data["Close"]:
                analysis[symbol] = "Recommendation: Sell"
            else:
                analysis[symbol] = "Recommendation: Hold"

    return analysis

def main():
    st.set_page_config(page_title="Stock Analysis App", page_icon=":chart_with_upwards_trend:")

    st.title("Stock Analysis App")
    st.write("Enter stock symbols separated by commas (e.g., AAPL, MSFT)")

    user_input = st.text_input("Enter stock symbols:")
    nyse_ticker_symbols = [symbol.strip().upper() for symbol in user_input.split(",")]

    if st.button("Fetch Real-Time Data"):
        try:
            realtime_data = get_realtime_data(nyse_ticker_symbols)

            for symbol, data in realtime_data.items():
                st.write(f"\n{symbol} Real-Time Data:")
                for key, value in data.items():
                    st.write(f"{key}: {value}")

                plot_ma_option = st.checkbox(f"Show 20-day and 50-day moving averages for {symbol}")
                plot_price_graph(symbol, show_ma=plot_ma_option)

            recommendations = analyze_data(realtime_data)
            st.write("\nAnalysis and Recommendations:")
            for symbol, recommendation in recommendations.items():
                st.write(f"{symbol}: {recommendation}")

        except Exception as e:
            st.error(f"Error fetching real-time data: {e}")

if __name__ == "__main__":
    main()



SyntaxError: ignored